In [1]:
import numpy as np
import pandas as pd
from utils.diff import diff

In [2]:
# 24. depr
# 半年频率。折旧除以固定资产。

# 53. pchdepr
# 半年频率。折旧变动百分比。

In [3]:
import pandas as pd
import numpy as np
from utils.diff import diff 

# 读取资产负债表
df1 = pd.read_csv('季_资产负债表.csv', usecols=['证券代码', '会计期间', '固定资产净额'], low_memory=False)

# 确保证券代码为整数类型
df1['证券代码'] = pd.to_numeric(df1['证券代码'], errors='coerce').fillna(0).astype(int)

# 读取现金流量表
df2 = pd.read_csv('季_现金流量表(间接法).csv', usecols=['证券代码', '会计期间', '固定资产折旧、油气资产折耗、生产性生物资产折旧', '查询成功'], low_memory=False)

# 确保证券代码为整数类型
df2['证券代码'] = pd.to_numeric(df2['证券代码'], errors='coerce').fillna(0).astype(int)

# 填充缺失值
df2.loc[df2['查询成功'].astype('bool'), '固定资产折旧、油气资产折耗、生产性生物资产折旧'] = df2['固定资产折旧、油气资产折耗、生产性生物资产折旧'].fillna(0)

# 合并数据
df3 = pd.merge(df1, df2, on=['证券代码', '会计期间'], how='left')

# 标记半年
df3['半年'] = df3['会计期间'].apply(lambda x: True if x[5:7] in ['06', '12'] else False)
df3 = df3[df3['半年']].reset_index(drop=True)

# 计算差异
df3 = diff(df3, col=['固定资产折旧、油气资产折耗、生产性生物资产折旧'], freq='半年')

# 计算折旧比率
df3['depr'] = df3['固定资产折旧、油气资产折耗、生产性生物资产折旧'] / df3['固定资产净额'].replace(0, np.nan)

# 计算变化率
df3['pchdepr'] = (df3['固定资产折旧、油气资产折耗、生产性生物资产折旧'] - df3.groupby('证券代码')['固定资产折旧、油气资产折耗、生产性生物资产折旧'].shift(1)) / df3.groupby('证券代码')['固定资产折旧、油气资产折耗、生产性生物资产折旧'].shift(1).replace(0, np.nan)

# 选择最终列并导出
df4 = df3[['证券代码', '会计期间', 'depr', 'pchdepr']]
df4.to_csv('半年_24_53.csv', index=False, encoding='utf-8-sig')

# 输出结果 DataFrame
print(df4)

          证券代码        会计期间      depr   pchdepr
0            1  2000-06-30  0.047606       NaN
1            1  2000-12-31  0.044944  0.044860
2            1  2001-06-30  0.047096  0.139446
3            1  2001-12-31  0.048491  0.051976
4            1  2002-06-30  0.047973  0.164755
...        ...         ...       ...       ...
282676  920118  2022-06-30       NaN       NaN
282677  920118  2022-12-31       NaN       NaN
282678  920118  2023-06-30       NaN       NaN
282679  920118  2023-12-31       NaN       NaN
282680  920118  2024-06-30  0.059376       NaN

[282681 rows x 4 columns]
